In [1]:
import json 
import os
import sys

In [2]:
import re
from typing import Dict, Any

def parse_source_file(range_str: str) -> Dict[str, Any]:
    pattern = r"^(.*)\((\d+),(\d+)-(\d+),(\d+)\)$"
    match = re.match(pattern, range_str)
    
    if match:
        file_path = match.group(1)
        start_line = int(match.group(2))
        start_col = int(match.group(3))
        end_line = int(match.group(4))
        end_col = int(match.group(5))
        
        return {
            "file_path": file_path,
            "position": {
                "start_line": start_line,
                "start_col": start_col,
                "end_line": end_line,
                "end_col": end_col
            }
        }
    else:
        raise ValueError("Invalid range string format")
    

In [3]:
dump_logs = [
    l[len("TAC>> DATASET "):].strip()
    for l in open("dump.dataset.log").readlines() if l.startswith("TAC>> DATASET ")
]

In [4]:
from tqdm.notebook import tqdm
parsed_logs = []
for dl in tqdm(dump_logs):
    try:
        d = json.loads(dl)
        fp = parse_source_file(d["range"])
        d["file_path"] = fp["file_path"]
        d["position"] = fp["position"]
        parsed_logs.append(d)
    except:
        print(dl)
        raise
    
with open("dump.dataset.json", "w") as f:
    json.dump(parsed_logs, f, indent=2)

  0%|          | 0/6900 [00:00<?, ?it/s]

In [5]:
taken = set()
taken_parsed = []
for p in parsed_logs:
    k = str(p)
    if k not in taken:
        taken.add(k)
        taken_parsed.append(p)

print(f"Removed {len(parsed_logs) - len(taken_parsed)} duplicates")

Removed 0 duplicates


In [6]:
# Sort the parsed logs by (file_path, start_line, start_col, end_line, end_col)
parsed_logs = sorted(
    parsed_logs, 
    key=lambda x: (x["file_path"], x["position"]["start_line"], x["position"]["start_col"], x["position"]["end_line"], x["position"]["end_col"])
)

In [7]:
node_id_to_dataset = {}
for didx, d in enumerate(parsed_logs):
    d["id"] = didx
    node_id_to_dataset[didx] = d

print(len(node_id_to_dataset))

6900


In [24]:
class Node:
    def __init__(self, nidx):
        self.nidx = nidx
        data_point = node_id_to_dataset[nidx]
        self.data_point = data_point
        self.file_name = data_point["file_path"]
        self.start_line = data_point["position"]["start_line"]
        self.end_line = data_point["position"]["end_line"]
        self.start_col = data_point["position"]["start_col"]
        self.end_col = data_point["position"]["end_col"]
        self.source_presence = data_point["is_in_source"]
        self.children = []
        self.parent = None
    
    def is_my_decendent(self, other):
        if other.file_name != self.file_name:
            return False
        if self.start_line == self.end_line and other.start_line == other.end_line and self.start_line == other.start_line:
            return self.start_col <= other.start_col and self.end_col >= other.end_col
        return self.start_line <= other.start_line and self.end_line >= other.end_line
    
    def add_child(self, child_node):
        for c in self.children:
            if c.is_my_decendent(child_node):
                c.add_child(child_node)
                return
        self.children.append(child_node)
        child_node.parent = self
        
    def __str__(self):
        statement = json.dumps((self.data_point['statement'][:30] + '...') if len(self.data_point['statement']) > 30 else self.data_point['statement'])
        precond = json.dumps(' -> '.join(self.data_point['preconditions'])[:30] + '...') if len(';'.join(self.data_point['preconditions'])) > 30 else ';'.join(self.data_point['preconditions'])
        postcod = json.dumps((str(self.data_point['postcondition'])[:30] + '...') if len(str(self.data_point['postcondition'])) > 30 else str(self.data_point['postcondition']))
        return f"{'✓' if self.source_presence else '✕'} {self.data_point['type_of_statement']} " +\
            f"({self.start_line},{self.start_col})-({self.end_line},{self.end_col}) : {statement} -> {precond} -> {postcod}"
    
    def prettyPrint(self, indent=0):
        print("|--" * indent + str(self))
        for c in self.children:
            c.prettyPrint(indent+1)
        
        
    def size(self):
        return 1 + sum([c.size() for c in self.children])
        
        

In [25]:
nodes = []
for didx in node_id_to_dataset:
    d = node_id_to_dataset[didx]
    if d["file_path"] != "lib/Pulse.Lib.HashTable.fst":
        continue
    d = node_id_to_dataset[didx]
    n = Node(didx)
    if len(nodes) == 0:
        nodes.append(n)
    else:
        for node in nodes:
            if node.is_my_decendent(n):
                node.add_child(n)
                break
        else:
            nodes.append(n)
            
for nidx, n in enumerate(nodes):
    print(f"Node {nidx}: ", n.size(), n)

Node 0:  14 ✓ totbind (57,2)-(63,4) : "let (contents: _) = Pulse.Lib...." -> "pure (FStar.SizeT.fits (2 * FS..." -> "(exists* (pht:Pulse.Lib.HashTa..."
Node 1:  4 ✓ proofhintwithbinders (75,10)-(76,15) : "unfold Pulse.Lib.HashTable.mod..." -> "(exists* (pht:Pulse.Lib.HashTa..." -> "emp"
Node 2:  2 ✓ return (77,2)-(77,20) : "Pulse.Lib.Vec.free (contents h..." -> "Pulse.Lib.Vec.pts_to ht.conten..." -> "emp"
Node 3:  118 ✓ totbind (100,2)-(196,5) : "let (hashf: _) = hashf ht;\nlet..." -> "Pulse.Lib.HashTable.models ht ..." -> "Pulse.Lib.HashTable.models (FS..."
Node 4:  47 ✓ totbind (217,2)-(241,3) : "let (hashf: _) = hashf ht;\nlet..." -> "Pulse.Lib.HashTable.models ht ..." -> "Pulse.Lib.HashTable.models (FS..."
Node 5:  193 ✓ totbind (262,2)-(413,3) : "let (hashf: _) = hashf ht;\nlet..." -> "Pulse.Lib.HashTable.models ht ..." -> "(exists* (pht':Pulse.Lib.HashT..."
Node 6:  55 ✓ totbind (438,2)-(489,3) : "let (hashf: _) = hashf ht;\nlet..." -> "Pulse.Lib.HashTable.models ht ..." -> "Puls

In [26]:
nodes[4].prettyPrint()

✓ totbind (217,2)-(241,3) : "let (hashf: _) = hashf ht;\nlet..." -> "Pulse.Lib.HashTable.models ht ..." -> "Pulse.Lib.HashTable.models (FS..."
|--✕ return (217,14)-(217,22) : "hashf ht" -> "Pulse.Lib.HashTable.models ht ..." -> ""
|--✓ withlocal (218,2)-(241,3) : "let mut (contents: _) = conten..." -> "pure (hashf == ht.hashf) -> Pu..." -> "Pulse.Lib.HashTable.models (FS..."
|--|--✓ proofhintwithbinders (218,33)-(219,15) : "unfold Pulse.Lib.HashTable.mod..." -> "Pulse.Lib.HashTable.models ht ..." -> "Pulse.Lib.HashTable.models (FS..."
|--|--|--✕ proofhintwithbinders (218,33)-(219,15) : "with (?kt:\nPrims.eqtype)\n(?vt:..." -> "Pulse.Lib.HashTable.models ht ..." -> "Pulse.Lib.HashTable.models (FS..."
|--|--|--|--✕ bind (218,33)-(219,15) : "let (_: Prims.unit) =\n  rewrit..." -> "Pulse.Lib.HashTable.models ht ..." -> "Pulse.Lib.HashTable.models (FS..."
|--|--|--|--|--✕ rewrite (218,33)-(219,15) : "rewrite\nPulse.Lib.HashTable.mo..." -> "Pulse.Lib.HashTable.models ht ..." -> ""
|--|--✓ pr